In [1]:
# Disable INFO and WARNING messages from TensorFlow
import os
os.environ['TF_CPP_MIN_LOG_LEVEL']='2'

import numpy as np
from imp import reload
from keras.layers import * # YL
from keras.models import Model
from keras.callbacks import ModelCheckpoint, EarlyStopping

# ============== DL ===============================
# Limit GPU memory(VRAM) usage in TensorFlow 2.0
# https://github.com/tensorflow/tensorflow/issues/34355
# https://medium.com/@starriet87/tensorflow-2-0-wanna-limit-gpu-memory-10ad474e2528

import tensorflow as tf

# ---- Method 1 ----
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)
# ============== DL ===============================
from utility.helper import psnr, load_imgs, image_to_block, regroup, save_imgs, performance_evaluation
from utility.parameter import *

import coarse.train as coarse_train
import coarse.test as coarse_test
import prediction.b1_train as prediction_train_b1
import prediction.b1_inference as prediction_inference_b1
import prediction.b23_train as prediction_train_b23
import prediction.b23_inference as prediction_inference_b23
import residue.b1_train as residue_train_b1
import residue.b23_train as residue_train_b23
import residue.b_inference as residue_final

# ============== DL ===============================
# Default is 1e-7 which is too small for float16.  
# Without adjusting the epsilon, we will get NaN predictions because of divide by zero problems
import keras.backend as K
if rtx_optimizer == True:
    K.set_epsilon(1e-4) 
# =================================================

b = 16 # blk_size
bm = 8 # target block size to predict

train_start, train_end = 0, 100
train_images = load_imgs(data_dir, train_start, train_end) 
n_train_frames = train_end - train_start

Using TensorFlow backend.


In [2]:
# ==== coarse train b1 ====
# coarse_train.model(train_images, b, training_ratio)

# ==== prediction train b1 ====
# decoded = coarse_test.predict(train_images, b, training_ratio)
# prediction_train_b1.pred_train(train_images, decoded, b, bm, training_ratio)

# ==== residue train b1 ====
# predicted_b1_frame = prediction_inference_b1.pred_inference_b1(decoded, b, bm, training_ratio)
# residue_train_b1.residue_train(train_images[2:n_train_frames - 2], predicted_b1_frame, bm, b, training_ratio)

# ==== prediction train b23 ====
# final_predicted_b1 = residue_final.residue_inference(train_images[2:n_train_frames-2], predicted_b1_frame, b, "residue_b1", training_ratio)
# prediction_train_b23.pred_train_b23(decoded, final_predicted_b1, train_images, b, bm, training_ratio)

# ==== residue train b23 ====
# predicted_b2_frame = prediction_inference_b23.pred_inference_b23( \
#         decoded[0:n_train_frames - 4], final_predicted_b1, b, bm, training_ratio)
# residue_train_b23.residue_train(train_images[1:n_train_frames - 3], predicted_b2_frame, bm, b, training_ratio)

In [3]:
# ---- coarse ----
test_images = load_imgs(data_dir, test_start, test_end)
decoded = coarse_test.predict(test_images, b, testing_ratio)

# n2_evaluated, coarse_amse, coarse_apsnr, coarse_assim = performance_evaluation(test_images, decoded, 0, 1)
# print('n_evaluated:',n2_evaluated)
# print('average test coarse_amse:',coarse_amse)
# print('average test coarse_apsnr:',coarse_apsnr)
# print('average test coarse_assim:',coarse_assim)

# save_imgs(save_dir, test_start, decoded)

Loaded model from C:/Users/danni/Documents/GitHub/SPIEcode-Daniel/models/BlowingBubbles_416x240_50/48/hdf5/coarse.hdf5


In [4]:
# ---- prediction b1 ----
predicted_b1_frame = prediction_inference_b1.pred_inference_b1(decoded, b, bm, testing_ratio)

# n_predicted1, amse1, apsnr1, assim1 = performance_evaluation(test_images[2:n_test_frames-2], predicted_b1_frame, 0, 4)
# print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
# print('n_b1:',n_predicted1)
# print('average test b1_amse:',amse1)
# print('average test b1_apsnr:',apsnr1)
# print('average test b1_assim:',assim1)

Loaded model from C:/Users/danni/Documents/GitHub/SPIEcode-Daniel/models/BlowingBubbles_416x240_50/48/hdf5/prediction.hdf5


In [5]:
# ---- residue b1 ----
final_predicted_b1 = residue_final.residue_inference( \
    test_images[2:n_test_frames-2], predicted_b1_frame, b, "residue_b1", testing_ratio)

# n_predicted1, amse1, apsnr1, assim1 = performance_evaluation(test_images[2:n_test_frames-2], final_predicted_b1, 0, 4)
# print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
# print('n_b1:',n_predicted1)
# print('average test b1_amse:',amse1)
# print('average test b1_apsnr:',apsnr1)
# print('average test b1_assim:',assim1)

Loaded model from C:/Users/danni/Documents/GitHub/SPIEcode-Daniel/models/BlowingBubbles_416x240_50/48/hdf5/residue_b1.hdf5


In [6]:
# ---- prediction b23 ----
predicted_b2_frame = prediction_inference_b23.pred_inference_b23( \
    decoded[0:n_test_frames - 4], final_predicted_b1, b, bm, testing_ratio)

n_predicted2, amse2, apsnr2, assim2 = performance_evaluation(test_images[1:n_test_frames-3], predicted_b2_frame, 0, 4)
print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
print('n_b2:',n_predicted2)
print('average test b2_amse:',amse2)
print('average test b2_apsnr:',apsnr2)
print('average test b2_assim:',assim2)

Loaded model from C:/Users/danni/Documents/GitHub/SPIEcode-Daniel/models/BlowingBubbles_416x240_50/48/hdf5/prediction_b23.hdf5
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
n_b2: 24
average test b2_amse: 472.2774942539896
average test b2_apsnr: 21.447607183732174
average test b2_assim: 0.30975099200321193


In [7]:
# ---- residue b23 ----
final_predicted_b2 = residue_final.residue_inference( \
    test_images[1:n_test_frames-3], predicted_b2_frame, b, "residue_b23", testing_ratio)

n_predicted2, amse2, apsnr2, assim2 = performance_evaluation(test_images[1:n_test_frames-3], final_predicted_b2, 0, 4)
print("vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv")
print('n_b2:',n_predicted2)
print('average test b2_amse:',amse2)
print('average test b2_apsnr:',apsnr2)
print('average test b2_assim:',assim2)

Loaded model from C:/Users/danni/Documents/GitHub/SPIEcode-Daniel/models/BlowingBubbles_416x240_50/48/hdf5/residue_b23.hdf5
vvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvvv
n_b2: 24
average test b2_amse: 409.81514407151775
average test b2_apsnr: 22.05997948331459
average test b2_assim: 0.37553588640742913


In [8]:
# print('average test overall_amse:', (pred_res_amse+coarse_amse)/2.0)
# print('average test overall_apsnr:', (pred_res_apsnr+coarse_apsnr)/2.0)
# print('average test overall_assim:', (pred_res_assim+coarse_assim)/2.0)